In [14]:
from pulp import *
import pandas as pd

### Надо определить:
* Поставщиков A1, A2, A3 с запасами a1, a2 , a3 
* Потребителей B1, B2, B3, B4, B5 с потребностями b1, b2, b3, b4, b5

### Вношу данные и присваиваю:

In [15]:
suppliers = ["A1", "A2", "A3"] 

a1, a2, a3 = 70, 50, 240 

supply = {"A1": a1, "A2": a2, "A3" : a3} 

In [17]:
consumers = ["B1", "B2", "B3", "B4"] 

b1, b2, b3, b4 = 100, 10, 70, 160

needs = {
    "B1": b1,
    "B2": b2,
    "B3": b3,
    "B4": b4,
}

In [18]:
supply, needs

({'A1': 70, 'A2': 50, 'A3': 240}, {'B1': 100, 'B2': 10, 'B3': 70, 'B4': 160})

#### Проверка на то, что транспортная задача закрытого типа:

In [19]:
if sum(supply.values()) == sum(needs.values()):
    print("Задача закрытый типа.")
else:
    print("Задача открытого типа.")

Задача открытого типа.


### Матрица тарифов:

In [20]:
tariffs = [
    [16, 18, 9, 1],
    [13, 16, 18, 9],
    [14, 23, 12, 7]]

tariffs = makeDict([suppliers, consumers], tariffs, 0)

### Задача минимизации

In [21]:
model = LpProblem("Distribution_Problem", LpMinimize)
model

Distribution_Problem:
MINIMIZE
None
VARIABLES

### Целевая функция:

In [22]:
Routes = [(w, b) for w in suppliers for b in consumers]
goods = LpVariable.dicts("Количество_товаров", (suppliers, consumers), 0, None, LpInteger)

model += (
    lpSum([goods[w][b] * tariffs[w][b] for (w, b) in Routes]),
    "Сумма_траспортировки"
)
model

Distribution_Problem:
MINIMIZE
16*Количество_товаров_A1_B1 + 18*Количество_товаров_A1_B2 + 9*Количество_товаров_A1_B3 + 1*Количество_товаров_A1_B4 + 13*Количество_товаров_A2_B1 + 16*Количество_товаров_A2_B2 + 18*Количество_товаров_A2_B3 + 9*Количество_товаров_A2_B4 + 14*Количество_товаров_A3_B1 + 23*Количество_товаров_A3_B2 + 12*Количество_товаров_A3_B3 + 7*Количество_товаров_A3_B4 + 0
VARIABLES
0 <= Количество_товаров_A1_B1 Integer
0 <= Количество_товаров_A1_B2 Integer
0 <= Количество_товаров_A1_B3 Integer
0 <= Количество_товаров_A1_B4 Integer
0 <= Количество_товаров_A2_B1 Integer
0 <= Количество_товаров_A2_B2 Integer
0 <= Количество_товаров_A2_B3 Integer
0 <= Количество_товаров_A2_B4 Integer
0 <= Количество_товаров_A3_B1 Integer
0 <= Количество_товаров_A3_B2 Integer
0 <= Количество_товаров_A3_B3 Integer
0 <= Количество_товаров_A3_B4 Integer

### Задаю, а затем добавляю в модель:
* Ограничения объема запаса у каждого поставщика
* Ограничения потребности у каждого потребителя

In [23]:
for w in suppliers:
    model += (
        lpSum([goods[w][b] for b in consumers]) <= supply[w],
        "Сумма_товаров_поставщиков_%s" % w
    )
    
for b in consumers:
    model += (
        lpSum([goods[w][b] for w in suppliers]) == needs[b],
        "Сумма_товаров_потребителей_%s" % b
    )

model   

Distribution_Problem:
MINIMIZE
16*Количество_товаров_A1_B1 + 18*Количество_товаров_A1_B2 + 9*Количество_товаров_A1_B3 + 1*Количество_товаров_A1_B4 + 13*Количество_товаров_A2_B1 + 16*Количество_товаров_A2_B2 + 18*Количество_товаров_A2_B3 + 9*Количество_товаров_A2_B4 + 14*Количество_товаров_A3_B1 + 23*Количество_товаров_A3_B2 + 12*Количество_товаров_A3_B3 + 7*Количество_товаров_A3_B4 + 0
SUBJECT TO
Сумма_товаров_поставщиков_A1: Количество_товаров_A1_B1
 + Количество_товаров_A1_B2 + Количество_товаров_A1_B3
 + Количество_товаров_A1_B4 <= 70

Сумма_товаров_поставщиков_A2: Количество_товаров_A2_B1
 + Количество_товаров_A2_B2 + Количество_товаров_A2_B3
 + Количество_товаров_A2_B4 <= 50

Сумма_товаров_поставщиков_A3: Количество_товаров_A3_B1
 + Количество_товаров_A3_B2 + Количество_товаров_A3_B3
 + Количество_товаров_A3_B4 <= 240

Сумма_товаров_потребителей_B1: Количество_товаров_A1_B1
 + Количество_товаров_A2_B1 + Количество_товаров_A3_B1 = 100

Сумма_товаров_потребителей_B2: Количество_това

### Результаты минимизации

In [24]:
status = model.solve()

In [25]:
print(f"Статус решения: {model.status}, {LpStatus[model.status]}")

for v in model.variables():
    print(v.name, "=", v.varValue)

print(f"Оптимальное значение целевой функции = {value(model.objective)}")

Статус решения: 1, Optimal
Количество_товаров_A1_B1 = 0.0
Количество_товаров_A1_B2 = 0.0
Количество_товаров_A1_B3 = 0.0
Количество_товаров_A1_B4 = 70.0
Количество_товаров_A2_B1 = 40.0
Количество_товаров_A2_B2 = 10.0
Количество_товаров_A2_B3 = 0.0
Количество_товаров_A2_B4 = 0.0
Количество_товаров_A3_B1 = 60.0
Количество_товаров_A3_B2 = 0.0
Количество_товаров_A3_B3 = 70.0
Количество_товаров_A3_B4 = 90.0
Оптимальное значение целевой функции = 3060.0


### Оптимальный план и стоимость перевозки груза:

In [27]:
obj = pd.Series([v.varValue for v in model.variables()])

arr = obj.values

reshaped_arr = arr.reshape((3, 4))

In [28]:
df = pd.DataFrame(reshaped_arr, index=suppliers, columns=consumers)
print(f"Оптимальная стоимость перевозки груза = {value(model.objective)}")
df

Оптимальная стоимость перевозки груза = 3060.0


,B1,B2,B3,B4
A1,0.0,0.0,0.0,70.0
A2,40.0,10.0,0.0,0.0
A3,60.0,0.0,70.0,90.0
